In [7]:
import os

secret_key = os.urandom(16)   # Generate 16-byte key (128 bits)
print(secret_key.hex())  # Converting a key to a string format

5f7972d647ab04a55ea4daa7c314bdbf


In [3]:
import re
from datetime import datetime

# Reading the source data file line by line
with open('db_data_credit_card_input.sql', 'r') as file:
  sql_data = file.readlines()

# Function to convert date format from 'MM/DD/YY' to 'YYYY-MM-DD'
def convert_date_format(date_str):
  try:
    old_format_date = datetime.strptime(date_str, '%m/%d/%y')
    new_format_date = old_format_date.strftime('%Y-%m-%d')
    return new_format_date
  except ValueError:
    return date_str

# Function to convert a string
def transform_sql_line(line):
  pattern = r"VALUES\s*\(\s*'([^']*)'\s*,\s*'([^']*)'\s*,\s*'([^']*)'\s*,\s*'([^']*)'\s*,\s*'([^']*)'\s*,\s*'([^']*)'\s*\);"
  def replace_values(match):
    id = match.group(1)
    iban = match.group(2)
    pan = match.group(3)
    pin = match.group(4)
    cvv = match.group(5)
    expiring_date = convert_date_format(match.group(6))
    return f"VALUES ('{id}', '{iban}', AES_ENCRYPT('{pan}', @encryption_key), AES_ENCRYPT('{pin}', @encryption_key), AES_ENCRYPT('{cvv}', @encryption_key), '{expiring_date}');"
  new_line = re.sub(pattern, replace_values, line)
  return new_line

# New file and write lines
with open('db_data_credit_card_output.sql', 'w') as file:
  for line in sql_data:
    new_line = transform_sql_line(line)
    file.write(new_line)
